<a href="https://colab.research.google.com/github/derianr32/tareas_SyS/blob/main/Ejercicio_2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Simulación de Sistemas LTI (Ejercicio 2.5)

Demostración de la Linealidad y la Invarianza en el Tiempo para los sistemas:

* $y[n] = x[n]/3 + 2x[n-1] - y[n-1]$
* $y[n] = \sum_{k=-\infty}^{n} x^2[k]$
* $y[n] = \text{median}(x[n])$ (sobre una ventana de tamaño 3).
* $y(t) = Ax(t) + B$

La simulación comprueba:
1.  **Linealidad:** Si $H\{a x_1 + b x_2\} - (a H\{x_1\} + b H\{x_2\}) \approx 0$
2.  **Invarianza en el Tiempo (IT):** Si $H\{x[n-n_0]\} - y[n-n_0] \approx 0$

In [1]:
import numpy as np
from scipy.signal import lfilter

# --- Función Auxiliar para Desplazamiento (Invarianza en el Tiempo) ---
def shift(x_in, n0):
  """Desplaza una señal discretizada x_in en n0 muestras."""
  x = np.array(x_in, dtype=float)
  result = np.zeros_like(x, dtype=float)
  # Asegura que el array tenga la misma longitud, rellenando con ceros
  if n0 > 0 and n0 < len(x): # Retraso (shift a la derecha)
    result[n0:] = x[:-n0]
  elif n0 < 0: # Adelanto (shift a la izquierda)
    n0_abs = abs(n0)
    if n0_abs < len(x):
      result[:n0] = x[n0_abs:]
  else:
    result = x.copy()
  return result

# --- Definición de Sistemas ---

# 1. SLIT: y[n] + y[n-1] = (1/3)x[n] + 2x[n-1]
def sistema_1_1(x):
  b = [1.0/3.0, 2.0] # Coeficientes de x
  a = [1.0, 1.0]      # Coeficientes de y
  return lfilter(b, a, x)

# 2. No Lineal (Acumulador Cuadrado): y[n] = sum(x[k]^2)
def sistema_2_1(x):
  return np.cumsum(x**2)

# 3. No Lineal (Mediana Móvil): y[n] = median(x[n]) (Ventana 3)
def sistema_3_1(x):
  x_padded = np.pad(x, (1, 1), mode='edge')
  y = np.zeros_like(x, dtype=float)
  for n in range(len(x)):
    y[n] = np.median(x_padded[n:n+3])
  return y

# 4. No Lineal (Bias Constante): y(t) = Ax(t) + B
A = 2.0
B = 5.0 # B distinto de cero rompe la linealidad
def sistema_4_1(x):
  return A * x + B

# --- Ejecución de las Pruebas ---

# Datos de prueba para Linealidad
x1, x2 = np.array([1.0, 2.0, -1.0]), np.array([3.0, 0.0, 1.0])
a, b = 2.0, 3.0
# Datos de prueba para Invarianza en el Tiempo (IT)
x_it = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
n0 = 2 # Retraso de 2

sistemas = {
    "1. SLIT (y[n]+y[n-1]...)": sistema_1_1,
    "2. No Lineal (sum x^2[k])": sistema_2_1,
    "3. No Lineal (Median W=3)": sistema_3_1,
    "4. No Lineal (Ax(t) + B)": sistema_4_1,
}

print("--- RESULTADOS DE LA SIMULACIÓN DE SISTEMAS (2.3) ---")
print("=====================================================")

for nombre, func in sistemas.items():
  print(f"\n### {nombre}")

  # 1. PRUEBA DE LINEALIDAD: H{a*x1 + b*x2} == a*H{x1} + b*H{x2}
  izq_lin = func(a*x1 + b*x2)
  der_lin = a * func(x1) + b * func(x2)
  error_lin = np.sum(np.abs(izq_lin - der_lin))

  print(f"  > Linealidad (Error): {error_lin:.2e}")
  if error_lin < 1e-9:
    print("  > Conclusión: SÍ es Lineal.")
  else:
    print("  > Conclusión: NO es Lineal (Error significativo).")

  # 2. PRUEBA DE INVARIANZA: H{x[n-n0]} == y[n-n0]
  izq_it = func(shift(x_it, n0))
  der_it = shift(func(x_it), n0)
  error_it = np.sum(np.abs(izq_it - der_it))

  print(f"  > Invarianza (Error): {error_it:.2e}")
  if error_it < 1e-9:
    print("  > Conclusión: SÍ es Invariante en el Tiempo (IT).")
  else:
    print("  > Conclusión: NO es Invariante en el Tiempo.")

--- RESULTADOS DE LA SIMULACIÓN DE SISTEMAS (2.3) ---

### 1. SLIT (y[n]+y[n-1]...)
  > Linealidad (Error): 7.11e-15
  > Conclusión: SÍ es Lineal.
  > Invarianza (Error): 0.00e+00
  > Conclusión: SÍ es Invariante en el Tiempo (IT).

### 2. No Lineal (sum x^2[k])
  > Linealidad (Error): 2.88e+02
  > Conclusión: NO es Lineal (Error significativo).
  > Invarianza (Error): 0.00e+00
  > Conclusión: SÍ es Invariante en el Tiempo (IT).

### 3. No Lineal (Median W=3)
  > Linealidad (Error): 1.00e+00
  > Conclusión: NO es Lineal (Error significativo).
  > Invarianza (Error): 0.00e+00
  > Conclusión: SÍ es Invariante en el Tiempo (IT).

### 4. No Lineal (Ax(t) + B)
  > Linealidad (Error): 6.00e+01
  > Conclusión: NO es Lineal (Error significativo).
  > Invarianza (Error): 1.00e+01
  > Conclusión: NO es Invariante en el Tiempo.
